# Supermarket customer analysis

**Georgios Zygoykis**  
**Ioannis Sakkis**  
Mining Big Datasets, MSc Business Analytics  
Department of Management Science and Technology  
Athens University of Economics and Business 

In [1]:
# Import libaries:
import numpy as np  # import numpy
import pandas as pd # import pandas
from sklearn.metrics import confusion_matrix
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

### 1) Import and pre-process the dataset with customers

After importing some basic libraries like pandas and numpy to work with our data , we had to import  data of customers that aquire groceries from our supermarket . Using functionalities like shape and head we get some insight of our customer's demographic characteristics .

In [2]:
groc = pd.read_csv('groceries.csv',sep=';')
print(groc.shape)
groc.head()

(10000, 10)


,Customer_ID,Age,Sex,Marital_Status,Education,Income,Customer_Rating,Persons_in_Household,Occupation,Groceries
0,1,75,male,married,primary,20000,very_good,3,retired,"citrus fruit,semi-finished bread,margarine,rea..."
1,2,61,female,single,secondary,28000,good,1,housemaid,"tropical fruit,yogurt,coffee"
2,3,32,male,single,secondary,34000,very_good,1,blue-collar,whole milk
3,4,62,male,married,primary,31000,very_good,3,blue-collar,"pip fruit,yogurt,cream cheese,meat spreads"
4,5,66,female,married,secondary,19000,good,3,retired,"other vegetables,whole milk,condensed milk,lon..."


Let's have a look at the data types of our dataframe . We can see that there are some issues , as culomns that should be numeric are not and there should be further investigation about it .

In [3]:
groc.dtypes

Customer_ID              int64
Age                     object
Sex                     object
Marital_Status          object
Education               object
Income                  object
Customer_Rating         object
Persons_in_Household     int64
Occupation              object
Groceries               object
dtype: object

Our initial thought was to check for missing values (NA's) . Below you can see that it looks like there are 0 NA's . Thus ,we decided to check those columns that have type issues element wised .

In [4]:
groc.isnull().sum()

Customer_ID             0
Age                     0
Sex                     0
Marital_Status          0
Education               0
Income                  0
Customer_Rating         0
Persons_in_Household    0
Occupation              0
Groceries               0
dtype: int64

Despite having zero NA's we had to check for wrong entries . After using unique function , we saw that there were some entries that were empty space . That space was missing data , so we had to deal with that later on . Below you can see the columns that had that issue , i.e. Age and Income . 

In [5]:
groc.Age.unique() # see space 

array(['75', '61', '32', '62', '66', '55', '23', '26', '29', '57', '28',
       '40', '53', ' ', '68', '33', '79', '54', '51', '60', '69', '47',
       '43', '46', '24', '65', '81', '82', '38', '44', '49', '48', '27',
       '35', '63', '80', '41', '78', '37', '21', '36', '71', '85', '34',
       '30', '50', '83', '64', '74', '70', '25', '73', '58', '76', '42',
       '67', '22', '52', '56', '84', '31', '45', '77', '86', '72', '39',
       '59'], dtype=object)

In [6]:
groc.Income.unique() # see space 

array(['20000', '28000', '34000', '31000', '19000', '35000', '21000',
       '30000', '32000', '26000', '14000', '13000', '9000', '25000', ' ',
       '16000', '18000', '23000', '39000', '41000', '40000', '10000',
       '33000', '8000', '29000', '37000', '36000', '27000', '22000',
       '38000', '17000', '42000', '24000', '44000', '48000', '45000',
       '51000', '46000', '12000', '54000', '43000', '53000', '15000',
       '3000', '47000', '7000', '56000', '50000', '49000', '11000',
       '55000', '4000', '6000', '52000', '57000', '5000', '1000', '64000',
       '58000', '2000', '61000', '60000', '0', '59000', '67000', '62000',
       '63000', '66000'], dtype=object)

In [7]:
groc.Persons_in_Household.unique() # ok

array([3, 1, 2, 4, 5, 6], dtype=int64)

In [8]:
#groc.Groceries.unique()

It high time we had a look on that space issue . Let's print some records that contain NA's for both columns . You can see that we miss that information , so we will impute those missing values with the mean value of the leftovers . 

In [9]:
groc.loc[groc.Income==' '].head(5)

,Customer_ID,Age,Sex,Marital_Status,Education,Income,Customer_Rating,Persons_in_Household,Occupation,Groceries
16,17,68,female,married,secondary,,good,3,retired,fruit/vegetable juice
27,28,43,male,single,tertiary,,poor,1,self-employed,"sausage,rolls/buns,soda,chocolate"
36,37,38,female,married,tertiary,,very_good,3,unemployed,fruit/vegetable juice
82,83,73,male,single,secondary,,very_good,1,retired,"root vegetables,onions,hard cheese,frozen vege..."
142,143,83,female,single,tertiary,,very_good,1,retired,"whole milk,frozen vegetables"


In [10]:
groc.loc[groc.Age==' '].head(5)

,Customer_ID,Age,Sex,Marital_Status,Education,Income,Customer_Rating,Persons_in_Household,Occupation,Groceries
15,16,,female,married,tertiary,25000,good,4,entrepreneur,"butter,sugar,fruit/vegetable juice,newspapers"
26,27,,female,married,primary,10000,excellent,3,housemaid,yogurt
60,61,,male,married,primary,37000,good,2,entrepreneur,"other vegetables,whole milk,frozen vegetables,..."
68,69,,female,single,secondary,27000,very_good,1,services,"sausage,whole milk,curd"
89,90,,female,divorced,secondary,23000,fair,1,housemaid,soda


We kept subsets for of the lefovers , one for records that contain Age and another one for those that contain income values . The porpuse was to turn those columns to numeric in order to calculate the mean values . 

In [11]:
groc_b = groc.loc[groc.Age!=' ']
groc_c = groc.loc[groc.Income!=' ']
#groc_d = groc.loc[groc.Persons_in_Household!=' ']

In [12]:
age_mean = round(pd.to_numeric(groc_b.Age).mean(),0)
inc_mean = round(pd.to_numeric(groc_c.Income).mean(),0)
#print(pd.to_numeric(groc.Persons_in_Household.mean()))

After computing those mean values , it was time to impute the missing values with their mean . The fact that culomn type was string , we had to replace the space with mean . 

In [13]:
groc['Income']=groc['Income'].replace([" "], inc_mean)
groc['Age']=groc['Age'].replace([" "], age_mean)

Now that have fixed the NA's , we have to change data types so as to finish our data preperation process .  

In [14]:
groc['Income'] = groc['Income'].astype('int64')
groc['Age'] = groc['Age'].astype('int64')
groc['Sex'] = groc['Sex'].astype('str')
groc['Education'] = groc['Education'].astype('str')
groc['Customer_Rating'] = groc['Customer_Rating'].astype('str')
groc['Occupation'] = groc['Occupation'].astype('str')
groc.dtypes

Customer_ID              int64
Age                      int64
Sex                     object
Marital_Status          object
Education               object
Income                   int64
Customer_Rating         object
Persons_in_Household     int64
Occupation              object
Groceries               object
dtype: object

In [15]:
#groc.dtypes

### 2) Compute data (dis-)similarity

At that stage of our assigment , we had to assess the similarity between customers . We want to form the **dissimilarity matrix** 
for all given attributes.
The most difficult part of this process is to compute similarities , or dissimilarities between sets . Before that we have to create a dictionary , containing groceriy purchases for each and every customer .

In [16]:
for i in range(len(groc)):
    groc.Groceries[i] = groc.Groceries[i].split(',')
   
myDict = {}
for i in range(len(groc)):
    myDict[i]= groc.Groceries[i]

There is an efficient method to compute simmilarities between sets . ***Jaccard distance*** , is a measure of how dis-similar two sets are.

In [17]:
# Jaccard distance for croceries 
#import itertools

#define jaccard similarity (size of intersection over size of union)
def jaccard_sim(list1, list2):
    intersection = len(list(set(list1).intersection(list2))) # why set ?
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

Now that we have the function to compute a similarity matrix of sets , we can compute a dissimilarity matrix too . Below you can see the code that can provide that functionality .

In [18]:
data1= [[0]*10000 for _ in range(10000)]
a=0
for i in myDict.keys():
    b=0
    for j in myDict.keys():
        data1[b][a]=1-jaccard_sim(myDict[i],myDict[j])
        b=b+1      
    a=a+1  
dis_jacr=np.tril(data1, 0)  #Convert into lower triangular matrix

Let's print the distance matrix as far as jaccard distance is concerned .

In [19]:
print(dis_jacr) # fix diangonal

[[0.         0.         0.         ... 0.         0.         0.        ]
 [1.         0.         0.         ... 0.         0.         0.        ]
 [1.         1.         0.         ... 0.         0.         0.        ]
 ...
 [1.         1.         1.         ... 0.         0.         0.        ]
 [0.90909091 0.77777778 1.         ... 1.         0.         0.        ]
 [1.         1.         1.         ... 0.8        1.         0.        ]]


Our next task is to compute dissimilarity matrix for ***nominal*** data . 

- Firstly , we have to create dictionaries again from columns contain nominal data . 

In [20]:
data_dict = groc['Sex'].to_dict()

In [21]:
data_dict1 = groc['Occupation'].to_dict()

In [22]:
data_dict2 = groc['Marital_Status'].to_dict()

Again using similar code to the one we used to compute the previous distance matrix , we create a function that takes as input a dictionary of nominal data , and composes a dissimilarity matrix .

In [23]:
def nominal_dis(data_dict) :
    data1= [[0]*10000 for _ in range(10000)]
    a=0
    for i in data_dict.keys():
        b=0
        for j in data_dict.keys():
            m=0       
            if data_dict[i]==data_dict[j]:
                m=m
            else:
                m=1-m
            data1[b][a]=m
            b=b+1      
        a=a+1  
    arr=np.tril(data1, 0)  #Convert into lower triangular matrix
    #print(arr)
    return(arr)

Below you can see the output for one distance matrix , concerning dictionaries with Nominal data .

In [24]:
dis_Gen = nominal_dis(data_dict)
dis_Occ = nominal_dis(data_dict1)
dis_Mar = nominal_dis(data_dict2)
print(dis_Occ)

[[0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 ...
 [1 1 0 ... 0 0 0]
 [1 0 1 ... 1 0 0]
 [1 1 1 ... 1 1 0]]


Our next task is to compute dissimilarity matrix for ***numerical*** data . 

- We have to create dictionaries again from columns contain numerical data .
- Compute a normalizer , to keep values in a scale between [0,1] .
- Create a function that composes a distance matrix .

In [25]:
normalizer=max(groc.Age)-min(groc.Age)

In [26]:
def Numerical_dis(data_dict):
    data1= [[0]*10000 for _ in range(10000)]
    a=0
    for i in data_dict.keys():
        b=0
        for j in data_dict.keys():
            m=abs(data_dict[i]-data_dict[j])/normalizer
            data1[b][a]=m
            b=b+1      
        a=a+1  
    arr=np.tril(data1, 0)  #Convert into lower triangular matrix
    return(arr)

- The following code creates a dissimilarity matrix for every Numerical attribute and prints the output on the console . 

In [27]:
data_dict3 = groc['Age'].to_dict()
dis_Age = Numerical_dis(data_dict3)
print(dis_Age)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.21538462 0.         0.         ... 0.         0.         0.        ]
 [0.66153846 0.44615385 0.         ... 0.         0.         0.        ]
 ...
 [0.56923077 0.35384615 0.09230769 ... 0.         0.         0.        ]
 [0.47692308 0.26153846 0.18461538 ... 0.09230769 0.         0.        ]
 [0.24615385 0.03076923 0.41538462 ... 0.32307692 0.23076923 0.        ]]


In [28]:
data_dict6 = groc['Persons_in_Household'].to_dict()
normalizer=max(groc.Persons_in_Household)-min(groc.Persons_in_Household)
dis_house = Numerical_dis(data_dict6)
print(dis_house)

[[0.  0.  0.  ... 0.  0.  0. ]
 [0.4 0.  0.  ... 0.  0.  0. ]
 [0.4 0.  0.  ... 0.  0.  0. ]
 ...
 [0.2 0.6 0.6 ... 0.  0.  0. ]
 [0.2 0.6 0.6 ... 0.  0.  0. ]
 [0.2 0.2 0.2 ... 0.4 0.4 0. ]]


In [29]:
data_dict7 = groc['Income'].to_dict()
normalizer=max(groc.Income)-min(groc.Income)
dis_inc = Numerical_dis(data_dict7)
print(dis_inc)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.11940299 0.         0.         ... 0.         0.         0.        ]
 [0.20895522 0.08955224 0.         ... 0.         0.         0.        ]
 ...
 [0.3880597  0.26865672 0.17910448 ... 0.         0.         0.        ]
 [0.04477612 0.07462687 0.1641791  ... 0.34328358 0.         0.        ]
 [0.01492537 0.10447761 0.19402985 ... 0.37313433 0.02985075 0.        ]]


Finally , we have to compute the dissimilarity matrix for ***ordinal data*** ,i.e. Customer Rating and Education . 
- We first check unique values of ordinal dat ,
- then , assign numerical values considering the order .
- At last , compose distance matrix , normalised . 

Check unique Rating values . 

In [30]:
groc.Customer_Rating.unique()

array(['very_good', 'good', 'fair', 'excellent', 'poor'], dtype=object)

This is the code we used to assign numerical values to Customer Rating for each customer respectively .

In [31]:
for i in range(len(groc.Customer_Rating)) :
    if groc.Customer_Rating[i] == 'excellent' :
        groc.Customer_Rating[i] = 5
    elif groc.Customer_Rating[i] == 'very_good' :
        groc.Customer_Rating[i] = 4 
    elif groc.Customer_Rating[i] == 'good' :
        groc.Customer_Rating[i] = 3
    elif groc.Customer_Rating[i] == 'fair' :
        groc.Customer_Rating[i] = 2
    else :
        groc.Customer_Rating[i] = 1


In [32]:
#groc.Customer_Rating

So now that we have assigned those numerical values , we can use the function we created before , so as to create the distance matrix . We have calculated again a normalizer , to normalise data distance between [0,1] . You can see the output of the matrix below . 

In [33]:
data_dict4 = groc['Customer_Rating'].to_dict()
normalizer=max(groc.Customer_Rating)-min(groc.Customer_Rating)
dis_rating = Numerical_dis(data_dict4)
print(dis_rating)

[[0.   0.   0.   ... 0.   0.   0.  ]
 [0.25 0.   0.   ... 0.   0.   0.  ]
 [0.   0.25 0.   ... 0.   0.   0.  ]
 ...
 [0.5  0.25 0.5  ... 0.   0.   0.  ]
 [0.75 0.5  0.75 ... 0.25 0.   0.  ]
 [0.25 0.   0.25 ... 0.25 0.5  0.  ]]


We did the same process for education  .

In [34]:
groc.Education.unique()

array(['primary', 'secondary', 'tertiary'], dtype=object)

In [35]:
for i in range(len(groc.Education)) :
    if groc.Education[i] == 'tertiary' :
        groc.Education[i] = 3
    elif groc.Education[i] == 'secondary' :
        groc.Education[i] = 2
    else :
        groc.Education[i] = 1

In [36]:
data_dict5 = groc['Education'].to_dict()
normalizer=max(groc.Education)-min(groc.Education)
dis_educ = Numerical_dis(data_dict5)
print(dis_educ)

[[0.  0.  0.  ... 0.  0.  0. ]
 [0.5 0.  0.  ... 0.  0.  0. ]
 [0.5 0.  0.  ... 0.  0.  0. ]
 ...
 [0.  0.5 0.5 ... 0.  0.  0. ]
 [0.5 0.  0.  ... 0.5 0.  0. ]
 [1.  0.5 0.5 ... 1.  0.5 0. ]]


Finally , we have calculated the distance between clients , computing the average distance for every datatype. 

In [37]:
dis_matrix = (dis_Occ + dis_Mar + dis_Gen + dis_Age + dis_rating + dis_educ + dis_house + dis_jacr + dis_inc)/9
print(dis_matrix)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.60942084 0.         0.         ... 0.         0.         0.        ]
 [0.53005485 0.42063401 0.         ... 0.         0.         0.        ]
 ...
 [0.40636561 0.66361143 0.43015691 ... 0.         0.         0.        ]
 [0.54231001 0.35710479 0.63319939 ... 0.4650657  0.         0.        ]
 [0.41234214 0.53724965 0.50660161 ... 0.46069014 0.51784666 0.        ]]


### 3) Nearest Neighbor (NN) search

The 3rd task is to create a function that takes as input the customer-id and return the list of her/his *10 nearest neighbors* (most similar), along with the corresponding *similarity score*.

The function ,takes as input the ID and the number of neighbors we want .Computes a list with the corresponding simmilarities , then creates a dataframe with the asked result .  

In [38]:
def NN_search(customer_id,k) :
    cust=dis_matrix[customer_id-1,:]+dis_matrix[:,customer_id-1]
    A = np.array(cust)
    k=k+1
    idx = np.argpartition(A, k)
    ids=idx[:k]+1
    similar=1-A[idx[:k]]
    dataset = pd.DataFrame()
    dataset['Customer_ID'] = ids.tolist()
    dataset['Similarity'] = similar.tolist()
    dataset = dataset[dataset.Customer_ID != customer_id]
    dataset=dataset.sort_values(by=['Similarity'],ascending=False)
    return (dataset)

- For customers **73, 563, 1603, 2200, 3703, 4263, 5300, 6129, 7800, 8555** we will print their 10 nearest neighbors .

In [39]:
cust_list=[73, 563, 1603, 2200, 3703, 4263, 5300, 6129, 7800, 8555]
for i in cust_list :
    print("Nearest Neighbors for customer {} , along with the corresponding similarities  : ".format(i))
    print(NN_search(i,10))
    print("")

Nearest Neighbors for customer 73 , along with the corresponding similarities  : 
    Customer_ID  Similarity
0          1846    0.877514
3          1291    0.870188
1          1203    0.856563
4          5881    0.853451
2          1627    0.851008
6          3953    0.847538
9          6904    0.844608
8          5922    0.843969
7          8881    0.841191
10         3623    0.839827

Nearest Neighbors for customer 563 , along with the corresponding similarities  : 
    Customer_ID  Similarity
0          3634    0.930833
1          6168    0.907195
2          2839    0.881535
4          6196    0.875469
5          2766    0.873912
8          8108    0.873555
6           559    0.870596
7          6929    0.868478
9          9578    0.866005
10         7202    0.865427

Nearest Neighbors for customer 1603 , along with the corresponding similarities  : 
    Customer_ID  Similarity
5          7345    0.868835
1          7335    0.860037
3           568    0.847255
0           109    0.

### 4) Customer rating prediction

We calculate the disimillarity matrix, excluding the customer rating metric.

In [40]:
dis_matrix_rat_ex = (dis_Occ + dis_Mar + dis_Gen + dis_Age + dis_educ + dis_house + dis_jacr + dis_inc)/8

With the Nearest Neighbors (rating excluded) method, we find the 10 nearest neighbors without taking into consideration teh customers rating metric.

In [41]:
def NN_search_rat_ex(customer_id,k) :
    cust=dis_matrix_rat_ex[customer_id-1,:]+dis_matrix_rat_ex[:,customer_id-1]
    A = np.array(cust)
    k=k+1
    idx = np.argpartition(A, k)
    ids=idx[:k]+1
    similar=1-A[idx[:k]]
    dataset = pd.DataFrame()
    dataset['Customer_ID'] = ids.tolist()
    dataset['Similarity'] = similar.tolist()
    dataset = dataset[dataset.Customer_ID != customer_id]
    dataset=dataset.sort_values(by=['Similarity'],ascending=False)
    return (dataset)

An example again with customer 73. You can see there is a difference in the list with the 10 nearest neighbors .

In [42]:
ten_nn = NN_search_rat_ex(73,10)
print(" New Nearest Neighbors :")
print(ten_nn)
print("")
print(" Ten Nearest Neighbors before exclusing Rating Metric : ")
print("")
print(NN_search(73,10))

 New Nearest Neighbors :
    Customer_ID  Similarity
2          1627    0.863634
5          1846    0.862203
3          3953    0.859730
0          1291    0.853961
1          2502    0.853564
6          5484    0.852107
7          5502    0.850115
9          1292    0.848364
8          7467    0.848134
10         5162    0.847040

 Ten Nearest Neighbors before exclusing Rating Metric : 

    Customer_ID  Similarity
0          1846    0.877514
3          1291    0.870188
1          1203    0.856563
4          5881    0.853451
2          1627    0.851008
6          3953    0.847538
9          6904    0.844608
8          5922    0.843969
7          8881    0.841191
10         3623    0.839827


In [43]:
ten_nn = pd.merge(ten_nn,groc[['Customer_ID','Customer_Rating']],on='Customer_ID',how='left')
ten_nn

,Customer_ID,Similarity,Customer_Rating
0,1627,0.863634,5
1,1846,0.862203,4
2,3953,0.859730,3
3,1291,0.853961,4
4,2502,0.853564,2
5,5484,0.852107,1
6,5502,0.850115,2
7,1292,0.848364,3
8,7467,0.848134,3
9,5162,0.847040,2


Every time we merge the nearest neighbors with our initial dataset in order to find their ratings. Then, in classification using mean as factor to classify customer rating, we simply calculate the mean of all ratings.

In [45]:
def mean_class(dataset):
    dataset = pd.merge(dataset,groc[['Customer_ID','Customer_Rating']],on='Customer_ID',how='left')
    return(round(dataset.Customer_Rating.mean()))

In classification with weighted average, we use the rank of customer's nearest neighbors , along with their similarity score as weight . So , below there is a function that classifies customers to a rating .

In [61]:
def wavg_class(dataset):
    dataset = pd.merge(dataset,groc[['Customer_ID','Customer_Rating']],on='Customer_ID',how='left')
    num= 0
    den= 0
    for i in range(len(ten_nn)):
        num += dataset.Similarity[i] * dataset.Customer_Rating[i]
        den += dataset.Similarity[i]    
    return(round(num/den))

We store the real rating in y_true list.

In [47]:
y_true = groc.Customer_Rating.head(50)

Predicted class, from average method, is stored in in y_avg list .

In [48]:
y_avg = []
for i in range(0,50):
    df = NN_search_rat_ex(i,10)
    rating = mean_class(df)
    y_avg.append(rating)

Predicted class, from weighted average method, is stored in in y_wavg list .

In [62]:
y_wavg = []
for i in range(0,50):
    df = NN_search_rat_ex(i,10)
    rating = wavg_class(df)
    y_wavg.append(rating)

We create a dataframe with the true ratings and the predicted ratings through average and weighted average methods accoridngly.

In [63]:
y_avg = pd.DataFrame(y_avg)
y_wavg = pd.DataFrame(y_wavg)

ratings = pd.concat([y_true, y_avg,y_wavg], axis=1)
ratings.columns.values[0] = "y_true"
ratings.columns.values[1] = "y_avg"
ratings.columns.values[2] = "y_wavg"

These are the Customer Ratings , along with their the corresponding encoding .

1 - poor  
2 - fair  
3 - good  
4 - very_good  
5 - excellent

   Thus , below you can see a Dataframe , named `ratings` with the true rating , the average rating prediction prediction and the weighted average prediction respectively .

In [64]:
ratings

,y_true,y_avg,y_wavg
0,4,3,3
1,3,3,3
2,4,4,4
3,4,4,4
4,3,2,3
5,4,3,3
6,3,3,3
7,3,3,3
8,4,3,3
9,3,3,3


We calculate the numerators for both cases, in order to find the mean prediction error.

In [66]:
num=0
wnum=0
for i in range(0,50):
    num += abs(ratings.y_true[i]-ratings.y_avg[i])
    wnum += abs(ratings.y_true[i]-ratings.y_wavg[i])
print('Mean Prediction Error with average:',num/50)
print('Mean Prediction Error with weighted average:',wnum/50)


Mean Prediction Error with average: 0.76
Mean Prediction Error with weighted average: 0.72


We conclude that, by using the weighted average method we can achieve a smaller mean prediction error and as a result we can infer that it is a more efficient method.